In [13]:
"""
Q37. 플로이드
https://acmicpc.net/problem/11404 
시간: 20:05~20:35
모든 도시의 쌍 (A, B)에 대해서 도시 A에서 B로 가는데 필요한 비용의 최솟값이므로 플로이드-워셜 알고리즘을 1번 돌리면 구할 수 있다.

시작 도시와 도착 도시를 연결하는 노선은 하나가 아닐 수 있다 -> 플로이드 탐색 시작 전에 입력값으로 주어진 비용을 graph에 추가할 때 이전 값보다
작은지 검증해주는 절차가 필요하다. 

시간복잡도가 O(n^3)이라서 그런지 실행 속도가 엄청 느리다. 
input()대신 sys.stdin.readline()을 써야함(기존 답지 4328ms에서 632ms로 줄임)
"""
from math import inf
import sys

cities = int(input())
bus = int(input())

# 2차원 그래프 생성. 초기값은 INF, 자기 자신과의 거리는 0으로 초기화
graph = [[inf for _ in range(cities + 1)] for _ in range(cities + 1)]
for i in range(1, cities + 1):
    for j in range(1, cities + 1):
        if i == j:
            graph[i][j] = 0

# 버스 노선 정보를 받아서 graph에 반영 (기존 비용보다 작은 지 검증)
for _ in range(bus):
    i, j, value = map(int, sys.stdin.readline().split())
    graph[i][j] = value if value < graph[i][j] else graph[i][j]


# 3중 for문으로 모든 노드 간 간선의 최소비용을 체크
for k in range(1, cities+1):
    for a in range(1, cities+1):
        for b in range(1, cities+1):
            # 모든 노드들에 대해 현재 노드를 거쳐서 가는 모든 N^2 개의 경로를 고려함
            graph[a][b] = min(graph[a][b], graph[a][k] + graph[k][b])

# 간선 비용이 inf인 경우를 제외하고 간선 비용들을 각 줄에 하나씩 출력
for i in range(1, cities+1):
    for j in range(1, cities+1):
        if graph[i][j] == inf:
            print(0, end=" ")
        else:
            print(graph[i][j], end=" ")
    print()

[inf, inf, inf, inf, inf, inf]
[inf, 0, 2, 3, 1, 10]
[inf, inf, 0, inf, 2, inf]
[inf, 8, inf, 0, 1, 1]
[inf, inf, inf, inf, 0, 3]
[inf, 7, 4, inf, inf, 0]
0 2 3 1 4 
12 0 15 2 5 
8 5 0 1 1 
10 7 13 0 3 
7 4 10 6 0 


In [29]:
"""
Q38. 정확한 순위 

학생 수가 n이라고 했을 때, 임의의 학생의 순위를 알 수 있는 조건은, 
학생들 간의 상대적인 등수 비교를 2차원 adjancency adj_list로 나타냈을 때 해당 학생이 다른 학생들과 연결된 간선이 자기 자신을 포함해 n-1개면 알 수 있다. 
ex. 총 6명이 있다고 했을 때, 내 위로 2명, 내 아래로 3명이 있다는 정보가 있으면 나는 3등이라는 걸 알 수 있음. 2+3 = 6-1

입력값으로 주어지는 순위 비교만으로는 부족하고, 
if 1 < 2 and 2 < 4, then 1 < 4 처럼 논리적 유추를 통해서 주어지는 간선 외에도 논리적 추론이 가능한 간선들을 전부 다 탐색해야 함.
쉬운 탐색을 위해 adj_matrix외에도 동일한 정보를 담은 adj list를 따로 생성함. 
"""
from math import inf

students, comparisons = map(int, input().split())

# adj matrix 생성
graph = [[inf] * (comparisons + 1) for _ in range(comparisons + 1)]
for i in range(1, comparisons + 1):
    graph[i][i] = 0 # 자기 자신과의 관계는 0으로 책정

# adj list 생성
adj_list = [[] for _ in range(comparisons + 1)]
for i in range(comparisons):
    a, b = map(int, input().split())
    adj_list[a].append((a, b))  

# queue처럼 역할을 할 set를 생성
queuelike = set()

# 탐색 시작
for i in range(1, comparisons + 1):
    for j in adj_list[i]:
        queuelike.add(j)

    while queuelike:
        a, b = queuelike.pop() # 성적 비교 결과 가져오기
        
        if graph[a][b] == inf:  # 미방문일 경우에는
            graph[a][b] = 1 # 비교 결과 1 저장. 단순 해당 방향의 관계가 성립한다는 표시이므로 1이 아니라 다른 inf가 아닌 값이어도 성립함
            graph[b][a] = 1 # 상대 관계의 간선은 양방향으로 존재하므로 반대 시점에서의 관계도 저장

            for p in adj_list[b]: # 노드 b에 연결되어 있는 다른 간선들을 모두 queue에 삽입  # adj_list[b] = [(4, 2), (4, 6)]
                queuelike.add((a, p[1]))

# 결과 출력
# 특정 row의 모든 element를 합했을 때 만약 상대 등수가 불명확한 학생이 존재한다면 해당 row의 어떤 값이 갱신되지 않고 inf로 남아있을 것이므로, 
# row의 합이 inf가 아닌 학생의 수를 반환하면 된다. 
result = 0
for i in range(1, students + 1):
    sum = 0
    for j in range(1, students + 1):
        sum += graph[i][j]
    if sum < inf:
        result += 1
print(result)
            

1


In [33]:
"""
Q39. 화성 탐사

솔루션:
N x N의 각 셀을 그래프의 노드로 간주하게 되면 1,1번 노드 -> N,N번 노드와의 거리를 구하는 문제가 된다. 
한 셀은 동서남북, 총 4개 방향으로 다른 셀과 연결될 수 있으므로 가장자리에 있는 경우만 조심하고, 
각 셀에 있는 값을 그 셀과 연결되어 있는 모든 간선들의 비용으로 간주하면 될 것이다. 

다만, 어차피 무조건 N,N의 노드와의 거리만이 필요한데, 1, 1노드와 다른 모든 노드들간의 거리를 모두 구하는 것은
비효율적인 것 같다..
"""
# test_case = int(input()) # 테스트케이스의 개수

import heapq

test_case = [
              [[5, 5, 4], [3, 9, 1], [3, 2, 7]],
              [[3, 7, 2, 0, 1], [2, 8, 0, 9, 1], [1, 2, 1, 8, 1], [9, 8, 9, 2, 0], [3, 6, 5, 1, 5]],
              [[9, 0, 5, 1, 1, 5, 3], [4, 1, 2, 1, 6, 5, 3], [0, 7, 6, 1, 6, 8, 5], [1, 1, 7, 8, 3, 2, 3], [9, 4, 0, 7, 6, 4, 1], [5, 8, 3, 2, 4, 8, 3], [7, 4, 8, 4, 8, 3, 4]]
            ]

dy = [-1, 1, 0, 0]
dx = [0, 0, -1, 1]

for map_ in test_case:
    width = int(input()) # 맵의 가로 길이
    # map_ = [map(int, input().split()) for _ in range(width)]  # 맵 입력
    visited = [[False for j in range(width)] for i in range(width)]
    heap = []
    target_y, target_x = width - 1, width - 1

    # y좌표, , x좌표, cost로 이루어진 리스트를 힙큐의 최초 원소로 삽입
    heapq.heappush(heap, [map_[0][0], 0, 0])
    visited[0][0] = True
    
    while heap:
        cost, y, x = heapq.heappop(heap) # 가장 작은 원소를 꺼냄
        if (y == target_y) and (x == target_x): # 도착지점일 경우 반복문을 빠져나온다
          break
        
        else: # 도착지점이 아닌 경우 현재 위치에서 연결된 미방문 노드들을 힙큐에 추가함
          for d in range(4):
             ny = y + dy[d]
             nx = x + dx[d]
             if (0 <= ny < width) and (0 <= nx < width) and not visited[ny][nx]:
                heapq.heappush(heap, [cost+map_[ny][nx], ny, nx])
                visited[ny][nx] = True
    print(cost)

20
19
36


In [45]:
"""
Q40. 숨바꼭질

시작점과 다른 모든 점들과의 거리 중 최대인 헛간을 찾아야 하므로 다익스트라를 사용한다.
간선의 weight는 1이라고 가정함. 
"""

import heapq
from math import inf

n, m = map(int, input().split())
min_distance = [inf for _ in range(n)] # 최소거리를 저장하는 리스트
edges = [[] for _ in range(n)] # 간선 정보를 저장하는 리스트 

for _ in range(m):
    node_1, node_2 = map(int, input().split())
    edges[node_1 - 1].append(node_2 - 1) # 간선이 항상 양방향이라고 명시되어 있으므로 방향을 바꿔서도 추가함
    edges[node_2 - 1].append(node_1 - 1) 

heap = []
heapq.heappush(heap, (0, 0)) # (거리, 노드 인덱스) 튜플로 추가
min_distance[0] = 0 # 시작지점의 자기자신과의 거리는 0으로 설정

while heap:
    dist, idx = heapq.heappop(heap)
    if min_distance[idx] < dist: # 이미 방문한 노드라면 스킵
        continue 
    for adj_node in edges[idx]: # 연결된 노드를 iterate함
        new_cost = dist + 1
        if new_cost < min_distance[adj_node]: 
            min_distance[adj_node] = new_cost 
            heapq.heappush(heap, (new_cost, adj_node))

# 반환하는 건 1. 숨는 노드 번호, 숨는 노드와 시작노드와의 거리, 같은 거리의 노드 개수
max_dist = 0
max_idx = 0
for idx, dist in enumerate(min_distance):
    if dist != inf:
        if dist > max_dist:
            max_dist = dist 
            max_idx = idx

# 노드 인덱스가 아니라 번호를 반환해야 하므로 인덱스에 +1을 해서 반환홤
print(max_idx+1, max_dist, min_distance.count(max_dist))

3 2 3
